In [1]:
import os
import time
import json
import pickle
import requests
import warnings
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

import dgl
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torchvision import datasets, transforms

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
warnings.filterwarnings('ignore')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Preprocess and save birdsnap and Food-101

In [2]:
normalize = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

In [3]:
bird_dataset = datasets.ImageFolder("/media/extra_storage/anirban/birdsnap/download/images", normalize)
dataloader = DataLoader(bird_dataset, batch_size=128, shuffle=True, drop_last=True,
                        collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))

In [ ]:
data = []

for d in tqdm(bird_dataset): data.append(d)

In [ ]:
with open("/media/extra_storage/anirban/birdsnap/bird_resized.pickle", "wb") as f:
    pickle.dump(data, f)

In [ ]:
with open("/media/extra_storage/anirban/birdsnap/bird_resized.pickle", "rb") as f:
    data = pickle.load(f)

## Lets do birdsnap

In [4]:
effi_mod = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)


effi_mod.classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(),
    nn.Dropout(0.2),
    nn.Linear(in_features=1280, out_features=500)
)

Using cache found in /home/anirban/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [5]:
# create model
all_train, all_val, all_test = [], [], []
time_taken, epochs_taken = [], []

start = time.time()
model = effi_mod
model.to(device)
loss_fcn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.NAdam(model.parameters(), lr=0.05, weight_decay=10e-5)


def accuracy(a, b):
    a = torch.cat(a)
    b = torch.cat(b)
    return (a == b).sum() / len(a)

for epoch in range(100):
    model.train()
    torch.cuda.synchronize()
    
    pred, actual = [], []

    for batch in tqdm(dataloader):
        features = batch[0].to(device)
        labels = batch[1].to(device)
        logits = model(features)
        loss = loss_fcn(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, prediction = logits.max(axis=1)
        pred.append(prediction)
        actual.append(labels)
        

    train_acc = accuracy(pred, actual)
    # val_acc = evaluate(model, features, labels.cuda(), val_mask.cuda())
    # if args.early_stop:
    #     if stopper.step(val_acc, model, epoch, args.print_less): break

    print(f"Epoch: {epoch:3d} | Train Acc: {train_acc:.4f}", end="")

# if not args.print_less : print()
# if args.early_stop: model.load_state_dict(torch.load('es_checkpoint.pt'))
# train_acc = evaluate(model, features, labels.cuda(), train_mask.cuda())
# val_acc = evaluate(model, features, labels.cuda(), val_mask.cuda())
# test_acc = evaluate(model, features, labels.cuda(), test_mask.cuda())
# best_epoch = epoch - 100

# all_train.append(round(train_acc, 4))
# all_val.append(round(val_acc, 4))
# all_test.append(round(test_acc, 4))
# time_taken.append(round(time.time() - start, 4))
# epochs_taken.append(best_epoch)

# if not args.print_less : print('-' * 100)
# print(f"Iteration: {i+1}/{args.n_iter} | Train Accuracy: {train_acc:.4f} | Val Accuracy: {val_acc:.4f} | Test Accuracy: {test_acc:.4f} | Time Taken: {time_taken[-1]}")
# if not args.print_less : print('-' * 100)


# all_train, all_val, all_test, time_taken, epochs_taken = np.array(all_train), np.array(all_val), np.array(all_test), np.array(time_taken), np.array(epochs_taken)

# print()
# print("=" * 100)
# print(f"Train Accuracy :     \t{all_train}")
# print(f"Validation Accuracy :\t{all_val}")
# print(f"Test Accuracy :      \t{all_test}")
# print(f"Time Taken :         \t{time_taken}")
# print(f"Epochs Taken :       \t{epochs_taken}")
# print(f"Average Train Accuracy :     \t{all_train.mean():.4f} ± {all_train.std():.3f}")
# print(f"Average Validation Accuracy :\t{all_val.mean():.4f} ± {all_val.std():.3f}")
# print(f"Average Test Accuracy :      \t{all_test.mean():.4f} ± {all_test.std():.3f}")
# print(f"Average Time Taken :         \t{time_taken.mean():.4f}")
# print(f"Average Epochs Taken :       \t{epochs_taken.mean():.0f}")
# print("=" * 100)

  0%|          | 0/311 [00:00<?, ?it/s]

Epoch:   0 | Train Acc: 0.0027

  0%|          | 0/311 [00:00<?, ?it/s]

Epoch:   1 | Train Acc: 0.0025

  0%|          | 0/311 [00:00<?, ?it/s]

Epoch:   2 | Train Acc: 0.0018

  0%|          | 0/311 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Split birdsnap

In [21]:
import os
import shutil
import random

In [18]:
with open("/media/extra_storage/anirban/birdsnap/test_images.txt", 'r') as f:
    tests = list(map(str.strip, f.readlines()[1:]))

In [28]:
src_root = "/media/extra_storage/anirban/birdsnap/splits/train/"
val_root = "/media/extra_storage/anirban/birdsnap/splits/val/"
test_root = "/media/extra_storage/anirban/birdsnap/splits/test/"

for folder in os.listdir(src_root):
    os.system(f"mkdir {val_root}/{folder}")
    os.system(f"mkdir {test_root}/{folder}")

In [30]:
src_root = "/media/extra_storage/anirban/birdsnap/splits/train/"
dst_root = "/media/extra_storage/anirban/birdsnap/splits/test/"

for image in tests:
    src = os.path.join(src_root, image)
    if os.path.exists(src):
        dst = os.path.join(dst_root, image)
        os.system(f"mv {src} {dst}")

In [37]:
src_root = "/media/extra_storage/anirban/birdsnap/splits/train/"
dst_root = "/media/extra_storage/anirban/birdsnap/splits/val/"

for folder in os.listdir(src_root):
    folder_path = os.path.join(src_root, folder)
    files = os.listdir(folder_path)
    sample = random.sample(files, 3)
    dst_path = os.path.join(dst_root, folder)
    for file in sample:
        src_path = os.path.join(folder_path, file)
        os.system(f"mv {src_path} {dst_path}")

In [45]:
# Makeup less test image classes

src_root = "/media/extra_storage/anirban/birdsnap/splits/train/"
dst_root = "/media/extra_storage/anirban/birdsnap/splits/test/"

for folder in os.listdir(dst_root):
    count = len(os.listdir(os.path.join(dst_root, folder)))
    if count < 3:
        n = 3 - count
        src_path = os.path.join(src_root, folder)
        dst_path = os.path.join(dst_root, folder)
        files = os.listdir(src_path)
        sample = random.sample(files, n)
        for file in sample:
            src = os.path.join(src_path, file)
            os.system(f"mv {src} {dst_path}")

In [46]:
dst_root = "/media/extra_storage/anirban/birdsnap/splits/test/"

d = dict()
for folder in os.listdir(dst_root):
    count = len(os.listdir(os.path.join(dst_root, folder)))
    if count in d: d[count] += 1
    else: d[count] = 1

d

{3: 182, 4: 175, 5: 143}

## Create Dataloader

In [7]:
normalize = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

In [3]:
bird_train_data = datasets.ImageFolder("/media/extra_storage/anirban/birdsnap/splits/train/", normalize)

In [4]:
cur_label = 0
data = []

for bird, label in tqdm(bird_train_data):
    if label != cur_label:
        with open(f"/media/extra_storage/anirban/birdsnap/dataloaders/class_{label}.pickle", 'wb') as f: pickle.dump(data, f)
        data = []
        cur_label = label
    data.append((bird, label))

  0%|          | 0/36390 [00:00<?, ?it/s]

In [10]:
with open(f"/media/extra_storage/anirban/birdsnap/dataloaders/class_500.pickle", 'wb') as f: pickle.dump(data, f)

In [2]:
class MyDataset(Dataset):
    def __init__(self, filename):
        with open(filename, "rb") as f:
            data = pickle.load(f)
        self._data = data

    def __getitem__(self, idx):
        return self._data[idx]

    def __len__(self):
        return len(self._data)

In [3]:
root = "/media/extra_storage/anirban/birdsnap/dataloaders/"

dataset_list = []
for file in tqdm(os.listdir(root)):
    file_path = os.path.join(root, file)
    dataset_list.append(MyDataset(file_path))

print(len(dataset_list))

  0%|          | 0/500 [00:00<?, ?it/s]

500


In [4]:
concat_data = ConcatDataset(dataset_list)
dataloader = DataLoader(concat_data, batch_size=128, shuffle=True, drop_last=False)

In [8]:
# Create Validation and test sets

bird_test_data = datasets.ImageFolder("/media/extra_storage/anirban/birdsnap/splits/test/", normalize)
bird_val_data = datasets.ImageFolder("/media/extra_storage/anirban/birdsnap/splits/val/", normalize)

In [9]:
data = []

for datapoint in tqdm(bird_test_data): data.append(datapoint)
with open(f"/media/extra_storage/anirban/birdsnap/dataloaders/test.pickle", 'wb') as f: pickle.dump(data, f)

data = []

for datapoint in tqdm(bird_val_data): data.append(datapoint)
with open(f"/media/extra_storage/anirban/birdsnap/dataloaders/val.pickle", 'wb') as f: pickle.dump(data, f)

  0%|          | 0/1960 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

## Train Model